<a href="https://colab.research.google.com/github/amsmith123/Deep_Learning/blob/main/Deep_Q_Learning_for_Lunar_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [7]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

  Using cached Shimmy-0.2.1-py3-none-any.whl (25 kB)
  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Using cached AutoROM.accept-rom-license-0.6.1.tar.gz (434 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=376a629b70c4879fa850115a9928ee9e86068ba9dffdced976396a3e3d32e729
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing

### Importing the libraries

In [3]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [4]:
class Network(nn.Module):
    def __init__(self, state_size, action_size, seed = 42):
        super(Network, self).__init__()
        # Initialize seed manually
        self.seed = torch.manual_seed(seed)
        # First fully connected layer (connects input layer to first hidden layer)
        self.fc1 = nn.Linear(state_size, 64)
        # Second fully connected layer (connects first hidden layer to second hidden layer)
        self.fc2 = nn.Linear(64, 64)
        # Third fully connected layer (connects second hidden layer to output layer)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, state):
        x = self.fc1(state)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.fc3(x)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 2 - Training the AI

### Setting up the environment

In [8]:
import gymnasium as gym
env = gym.make(
    "LunarLander-v2",
    # continuous: bool = False,
    # gravity: float = -10.0,
    # enable_wind: bool = False,
    # wind_power: float = 15.0,
    # turbulence_power: float = 1.5,
)

state_shape = env.observation_space.shape
state_size = state_shape[0]
number_actions = env.action_space.n
# print(f'State Shape: {state_shape}')
# print(f'State Size: {state_size}')
# print(f'Number of Actions: {number_actions}')

### Initializing the hyperparameters

In [12]:
learning_rate = 5e-4
minibatch_size = 100
# discount_factor
gamma = 0.99
replay_buffer_size = int(1e5)
# interpolation_parameter
tau = 1e-3

### Implementing Experience Replay

In [9]:
class ReplayMemory(object):
    def __init__(self, capacity):
      self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      self.capacity = capacity
      self.memory = []

    def push(self,event):
      self.memory.append(event)
      if len(self.memory) > self.capacity:
        del self.memory[0]

    def sample(self, batch_size):
      experiences = random.sample(self.memory, k = batch_size)
      states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
      actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
      rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
      next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
      dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
      return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [10]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done) :
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1 ) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(minibatch_size)
        self.learn(experiences, gamma)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, gamma):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (gamma * next_q_targets * (1 - dones))
    q_expected =  self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork,tau)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)


### Initializing the DQN agent

In [13]:
agent = Agent(state_size,number_actions)

### Training the DQN agent

In [19]:
number_episodes = 2000
max_tstep = 1000
epsilon_starting_val = 1.0
epsilon_ending_val = 0.01
epsilon_decay_val = 0.995
epsilon = epsilon_starting_val
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1,number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(0, max_tstep):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_val, epsilon_decay_val * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 220.0:
    print('\rEnvioronment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -128.97
Episode 200	Average Score: -93.82
Episode 300	Average Score: -61.14
Episode 400	Average Score: 92.67
Episode 500	Average Score: 212.50
Envioronment solved in 420 episodes!	Average Score: 220.10


## Part 3 - Visualizing the results

In [21]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

# Load the saved model weights
checkpoint = torch.load('checkpoint.pth')
agent.local_qnetwork.load_state_dict(checkpoint)

# Set the network to evaluation mode
agent.local_qnetwork.eval()

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()